In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_tensor_type(torch.DoubleTensor)

import matplotlib.pyplot as plt 
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu"  


## Orthogonal greedy algorithm for finite neuron method: $L^2$-fitting


**OGA:**
\begin{equation} 
			f_0 = 0, \quad g_n  = \mathop{\arg \max}_{g \in \mathbb{D}}  | \langle g, f - f_{n-1}
			\rangle |, \quad f_n = P_n(f),  
		\end{equation}
		where $P_n$ is a projection onto $H_n = \text{span}\{g_1, g_2,...,g_n \}$
		 For 1D $L^2$-fitting for target $f(x)$ with $f(0) = 0$, the ReLU shallow neural network dictionary can be given by 
		\begin{equation}
			\mathbb{D} = \{ \sigma( x + b), b \in [-\pi,\pi]  \}, ~~~ \sigma(x) = \max(0,x)
		\end{equation}
        
<br>
<br>

**Theretically guaranteed convergence rate:** 

Let the iterates $f_n$ be given by the orthogonal greedy algorithm, where $f\in \mathcal{K}_1(\mathbb{P}_k^d)$, where $\mathbb{P}_k^d$ is the $\text{ReLU}^k$ dictionary, $\sigma_k(\omega \cdot x + b)$,  $\sigma(x) = \max^k(0,x)$

Then we have
  \begin{equation}
   \|f_n - f\|_{L^2} \lesssim \|f\|_{\mathcal{K}_1(\mathbb{P}_k^d)}n^{-\frac{1}{2} - \frac{2k+1}{2d}}.
  \end{equation}
  
 **In particular**, for a ReLU shallow neural network in 1D, we have ($k=1, d=1$)
   \begin{equation}
   \|f_n - f\|_{L^2} \lesssim \|f\|_{\mathcal{K}_1(\mathbb{P}_1^1)}n^{-2}.
  \end{equation}
  
<br>
<br>

**Two major steps in OGA**:

1. argmax 
2. projection

<br>
<br>

**How to implement the two major steps in OGA?**

1. Solve the argmax problem $\quad g_n  = \mathop{\arg \max}_{g \in \mathbb{D}}  | \langle g, f - f_{n-1}
			\rangle | $ 
    - $L^2$ innner product: numerical integration 
        - Piecewise Gauss quadrature (very accurate numerical quadrature is needed for this part)
    - **Method of exhaustion** for 1D for a good initial guess 
        - $\mathbb{D}_N = \{ \sigma(x + b_i), b_i = \pi -  2 \pi\frac{i-1}{N} \}_{i = 1}^N$
        - $\quad g_n  = \mathop{\arg \max}_{g \in \mathbb{D_N}}  | \langle g, f - f_{n-1}
			\rangle | $ 
            - store values of $g$ as a row vector, values of $f - f_{n-1}$ column vector

    

In [ ]:
## 1. ReLU dictionary discretization 
# relu dictionary
def relu_dict(x_l,x_r,N):

    relu_dict_parameters = torch.zeros((N,2))
    relu_dict_parameters[:N,0] = torch.ones(N)[:]
    relu_dict_parameters[:N,1] = torch.linspace(x_l,x_r,N+1)[1:] # relu(x+bi)

    return relu_dict_parameters
def target(x):
    return x 

## 2. Method of exhaustion 
relu_dict_parameters = relu_dict(-pi,pi,100)
integration_intervals = 10 
nodes = torch.linspace(-pi,pi,integration_intervals+1).view(-1,1) 
coef1 = ((nodes[1:,:] - nodes[:-1,:])/2) # n by 1  
coef2 = ((nodes[1:,:] + nodes[:-1,:])/2) # n by 1  
coef2_expand = coef2.expand(-1,gx.size(1)) # Expand to n by p shape, -1: keep the first dimension n , expand the 2nd dim (columns)
integration_points = coef1@gx + coef2_expand
integration_points = integration_points.flatten().view(-1,1) # Make it a column vector
gw_expand = torch.tile(gw,(integration_intervals,1)) # rows: n copies of current tensor, columns: 1 copy, no change
coef1_expand = coef1.expand(coef1.size(0),gx.size(1))    
coef1_expand = coef1_expand.flatten().view(-1,1)
func_values = target(integration_points) 

weight_func_values = func_values*gw_expand*coef1_expand
print(weight_func_values.size())
basis_values = F.relu(relu_dict_parameters[:,0] *integration_points + relu_dict_parameters[:,1]).T # uses broadcasting  
print(basis_values.size())
output = torch.abs(torch.matmul(basis_values,weight_func_values)) # each component contains an integral value 
print(output.size())
neuron_index = torch.argmax(output.flatten())

print(neuron_index)


2. Orthogonal projection. Given $H_n = \text{span}\{g_1, g_2,...,g_n \}$
    - Use w_list, b_list to keep track of the added neurons
        - Construct the neural network from the two lists 
    - Orthogonal projection: Solve the linear system.
         - $f_n = P_{H_m} f$  
         - $<f_n,g_i> = <f,g_i>$ for all $i= 1,2,...,n$, with $f_n = \sum_{i=1}^n a_i g_i $
         - A linear system in $\alpha := (a_1, a_2,...,a_n)^T$. $M \alpha = b$, where $M_{i,j} = <g_j,g_i>$ $b_i = <f,g_i> $
         - Equivalently, an $L^2$-fitting: $$\min_{a_1, a_2,...,a_n} \frac{1}{2} \|\sum_{i=1}^n a_i g_i  - f \|^2_{L^2}, $$ that is, $$\min_{\alpha \in \mathbb{R}^n} \frac{1}{2} \alpha^T M \alpha - b^T \alpha $$
    - A shortcut: make use of pytorch automatic differentiation to get the matrix M, since we already know how to compute the loss function very accurately.



#### Extract the linear system using pytorch auto differentiation and solve the linear system 

$L(x) = \frac{1}{2}x^T A x$

$\nabla_x L(x) = A x$

$\nabla_x (Ax)_i = \text{ ith row of } A $

In [2]:
def minimize_linear_layer(model,target,solver="cg"):
    """Solve the linear layer problem Mx = b: L2 fitting relu shallow neural networks 
    Parameters
    ----------
    model : 
        relu shallow neural network
    target: 
        a target function 
        
    Returns
    -------
    sol: tensor 
        the solution of the linear system 
    """
    def loss_function_inside(x):
        return 0.5*torch.pow(model(x)-target(x),2).to(device)

    def rhs_loss_inside(x): 
        return model(x)*target(x)
    #1. Compute loss function using piecewise Gaussian quadrature
    node = compute_integration_nodes_relunn(-pi,pi,model)
    loss = GQ_piecewise(gw,gx,node,loss_function_inside) #loss_function

    #2. Extract the linear system A using torch.autograd 
    du1 = torch.autograd.grad(outputs=loss, inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
    jac = '' 
    neuron_number = model.fc1.bias.size(0)
    for i in range(neuron_number): 
        duui = torch.autograd.grad(outputs=du1[0,i], inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
        if i == 0: 
            jac = duui
        else: 
            jac = torch.cat([jac,duui],dim=0)

    #3. Extract the right hand side
    loss_helper = GQ_piecewise(gw,gx,node,rhs_loss_inside)
    rhs = torch.autograd.grad(outputs=loss_helper, inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
    rhs = rhs.view(-1,1)

    #4. Solve the linear system 
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach()),np.array(rhs.detach()),tol=1e-10) 
    elif solver == "direct": 
        sol = np.linalg.inv( np.array(jac.detach()) )@np.array(rhs.detach())
    sol = torch.tensor(sol).view(1,-1)
    return sol 
